# De-Americanization Algorithm
------------------
> **Idriss Afra**

This project aims to imply the dividends and volatilities for American stocks and indices.

## US Stock & Index Options
Market-listed options on U.S. stocks and indices are usually American options, meaning they can be exercised on any business day before expiration. Since call-put parity doesn't apply to American options, it can't be used to derive forward and dividend curves. To overcome this limitation, various de-Americanization methods have been created, with the simplest and most common approach being the following (for each listed expiry):

Init :
- $n_{max}$, the maximum number of iterations
-   $F_0 = F(d_0)$, $d_0$ an initial dividend value guess and $F_0$ the initial forward value guess
- $K$, the strike of market call and put American options with the highest trading volume

While $n < n_{max}$ :
- Compute the implied American volatilities : $σ_{c}^{Amr}(K, F(d_k))$, $σ_{p}^{Amr}(K, F(d_k))$
- Compute the equivalent European prices : $C_{Eur}(K, σ_{c}^{Amr}(K, F(d_k)))$, $P_{Eur}(K, σ_{p}^{Amr}(K, F(d_k)))$
- Compute the forward using the call-put parity :

$$F_{k+1} = e^{rT} \left(C_{Eur}(K, σ_{c}^{Amr}(K, F(d_k))) - P_{Eur}(K, σ_{p}^{Amr}(K, F(d_k)))\right) + K $$

- Imply the corresponding dividend yield : $d_{k+1} = ln\left(\frac{Spot × e^{rT}}{F_{k+1}}\right) \div T$
- If :  $|F(d_k+1) - F(d_{k})| > ϵ$, Continue. Else : Break.

In this project, we use the Binomial model to imply American volatilities and price equivalent European options.

## The Binomial Model

The Binomial model is relatively easy to understand and implement as it assumes that the underlying asset $S_t$ moves up or down by a fixed percentage in each period.

Let's define the movement factors $u$ and $d$ as following :
$$
u = e^{\sigma\sqrt{\Delta t}}\qquad d = e^{-\sigma\sqrt{\Delta t}} \\
$$
Where $\sigma$ is the volatility and $Δt$ is a tiny time step to maturity $T$. The risk-neutral probability under this model is :

$$
p = \frac{e^{(r-q)\Delta t}-d}{u-d} \\
$$
So that :
$$
E(S_{t+\Delta t}) = p × u × S_t + (1 - p) × d × S_t  
= p × S_{t+\Delta t}^u + (1 - p) × S_{t+\Delta t}^d \\
$$

Where $r$ is the zero-coupon interest rate and $q$ the dividend yield.

Once the Binomial tree is simulated, the options are priced using a backward method :

* European Style :  
$$
PV_t(S_t) = e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)
$$

* American Style :    
$$
PV_t(S_t) = max\left(S_t - K, e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)\right) \\
$$

With the final condition : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right)  $

Where $ϕ = 1$ for call options and $ϕ = -1$ for put options, and $K$ is the strike price.

The accuracy of the binomial model improves with the number of time steps. However, this also raises the complexity and computation time.

In [1]:
import numpy as np
import sys

class binomial_model:
    """
    The Binomial model class.
    """
    def __init__(self, n_steps=750):
        """
        Init method.
        n_steps is the number of time steps.
        """
        self.n_steps = n_steps
    
    def spot_simulation(self, S0, T, r, sigma):
        """
        The spot simulation under the Binomial model.
        The function returns the Binomial tree.
        """
        dt = T / self.n_steps
        u = np.exp(sigma * np.sqrt(dt))
        d = 1. / u

        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d
        return s
    
    def binomial_price(self, S0, T, r, q, K, sigma, isCall, isAmerican=True):
        """
        The Binomial American / European price.
        """
        # Spot simulation
        dt = T / self.n_steps
        u = np.exp(sigma * np.sqrt(dt))
        d = 1. / u
        p = (np.exp((r - q) * dt) - d) / (u - d)
        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d

        # Option payoff
        phi = 1 if isCall else -1
        v = np.maximum(phi * (s[:, self.n_steps] - K), 0.) # payoff at final date T

        # Discount between 2 time steps
        discount = np.exp(-r * dt)

        # Backward loop
        for i in range(self.n_steps-1, -1, -1) : # => i = n_steps-1 ... 0
            n_nodes = i+1 # i+1 nodes at time step #i
            v = discount * ( p * v[:n_nodes] + (1-p) * v[1:n_nodes+1] )
            if isAmerican :
                v = np.maximum(phi * (s[:n_nodes, i] - K), v)
        return v[0]
    
    def implied_vol(self, S0, T, r, q, K, isCall, market_price, isAmerican=True, sigma_min=0.001, sigma_max=10, n_max=750, eps=1e-8):
        """
        Bisection algorithm under the Binomial model.
        The function returns the implied American / European vol.
        """
        n = 1
        while n <= n_max :
            sigma = (sigma_min + sigma_max)  / 2
            f_sigma = self.binomial_price(S0, T, r, q, K, sigma, isCall, isAmerican) - market_price
            if (f_sigma == 0) or (sigma_max - sigma_min < eps) :
                return sigma
            if f_sigma < 0 :
                sigma_min = sigma
            else :
                sigma_max = sigma
            n += 1
        return sys.exit("The volatility calibration algorithm failed to converge. Please review data of the " + flavor.upper() + " with Strike " + str(K))

In [2]:
# Test
tree_model = binomial_model(n_steps=3)
tree_model.spot_simulation(100, 1,0.03, 0.2)

array([[100.        , 112.24009024, 125.97837858, 141.39824581],
       [  0.        ,  89.09472523, 100.        , 112.24009024],
       [  0.        ,   0.        ,  79.37870064,  89.09472523],
       [  0.        ,   0.        ,   0.        ,  70.72223522]])

In [3]:
# Test the Binomila pricing
binomial_model = binomial_model()
print("Example of a Put option on stock with dividends :")
print("-------------------------------------------------")
print("American Put price", round(binomial_model.binomial_price(105, 1, 0.03, 0.01, 100, 0.2, False), 4))  
print("European Put price", round(binomial_model.binomial_price(105, 1, 0.03, 0.01, 100, 0.2, False, False), 4))  

Example of a Put option on stock with dividends :
-------------------------------------------------
American Put price 5.1462
European Put price 5.0191


## De-Americanization Algorithm

In [4]:
class deAmericanization_method:
    """
    The de-Americanization method class :
    1- Call the implied_vol function to calibrate volatilities of the American call and put options with the highest trading volume (American Binomial model)
    2- Use these volatilities to re-price the equivalent European call and put prices (European Binomial model)
    3- Imply the forward using the call-put parity
    4- Re-do the 1st, 2nd, and 3rd steps until the convergence of the implied forward
    """
    def __init__(self, binomial_model, n_max=750, eps=1e-6):
        """
        Init method.
        """
        self.binomial_price = binomial_model.binomial_price
        self.amr_implied_vol = binomial_model.implied_vol
        self.n_max = n_max
        self.eps = eps
        
    def implied_forward(self, S0, T, r, q_init, strike, amer_call_price, amer_put_price) :
        """
        This function returns the implied forward.
        """
        sigma_call = self.amr_implied_vol(S0, T, r, q_init, strike, True, amer_call_price)
        sigma_put = self.amr_implied_vol(S0, T, r, q_init, strike, False, amer_put_price)
        eur_price_call = self.binomial_price(S0, T, r, q_init, strike, sigma_call, True, False)
        eur_price_put = self.binomial_price(S0, T, r, q_init, strike, sigma_put, False, False)
        F = np.exp(r * T) * (eur_price_call - eur_price_put) + strike
        q = np.log(S0 / F) / T + r
        nb_iteration = 1

        print("-------------------------------------------------------")
        print("Step : ", nb_iteration)
        print("Implied Forward : ", round(F, 8))
        print("Implied Dividend Yield : " + str(round(100 * q, 8)) + "%")

        while nb_iteration < self.n_max :

            sigma_call = self.amr_implied_vol(S0, T, r, q, strike, True, amer_call_price)
            sigma_put = self.amr_implied_vol(S0, T, r, q, strike, False, amer_put_price)

            print("Market American Call Price : ", round(amer_call_price, 8))
            print("Binomial American Call Price : ", round(self.binomial_price(S0, T, r, q, strike, sigma_call, True), 8))
            print("Implied American Call Vol : " + str(round(100 * sigma_call, 8)) + "%")
            print("Market American Put Price : ", round(amer_put_price, 8))
            print("Binomial American Put Price : ", round(self.binomial_price(S0, T, r, q, strike, sigma_put, False), 8))
            print("Implied American Put Vol : " + str(round(100 * sigma_put, 8)) + "%")

            eur_price_call = self.binomial_price(S0, T, r, q, strike, sigma_call, True, False)
            eur_price_put = self.binomial_price(S0, T, r, q, strike, sigma_put, False, False)

            print("Equivalent Binomial European Call Price : ", round(eur_price_call, 8))
            print("Equivalent Binomial European Put Price : ", round(eur_price_put, 8))

            old_F = F
            F = np.exp(r * T) * (eur_price_call - eur_price_put) + strike
            q = np.log(S0 / F) / T + r

            if abs(F - old_F) < self.eps :
                return F

            nb_iteration += 1

            print("-------------------------------------------------------")
            print("De-Americanization Results :")
            print("Step : ", nb_iteration)
            print("Implied Forward : ", round(F, 8))
            print("Implied Dividend Yield: " + str(round(100 * q, 8)) + "%")

        return sys.exit("The Dividends calibration algorithm failed to converge. Please review the inputs")


## Application : AAPL US Listed Equity Options

Let's apply the de-Americanization algorithm outlined above to the Apple US stock options as of August 10, 2023. If the AAPL US repo rates are not zero, they will be incorporated into the implied dividend.

In [5]:
from datetime import date
import matplotlib.pyplot as plt

t0 = date(2023, 8, 10)
S0 = 177.83
r = 5.482 / 100
exp = date(2023, 12, 15)
T = (exp - t0).days / 365
# OTM listed American calls and puts prices
strikes = [165, 170, 175, 180, 185, 190]
amer_prices = [4.425, 5.75, 7.425, 10.125, 7.625, 5.5250]
isCall = [False] * 3 + [True] * 3

# The highest trading volume strike asof 8 Aug 2023 is 185. Its associated call option close price is 7.625 and its put option close price is 11.975
q_init = 0.0001
print("Dividend Yield Initial Guess : " + str(round(100 * q_init, 4)) + "%")
deAmericanization_method = deAmericanization_method(binomial_model)
fwd = deAmericanization_method.implied_forward(S0, T, r, q_init, 185, 7.625, 11.975)
div_yield = np.log(S0 / fwd) / T + r
div_cash = S0 * np.exp(r * T) - fwd

print("-------------------------------------------------------")
print("Implied Fwd : ", round(fwd, 2))
print("Implied Dividend Yield : " + str(round(100 * div_yield, 2)) + "%")
print("Implied Cash Dividend : ", round(div_cash, 2))

Dividend Yield Initial Guess : 0.01%
-------------------------------------------------------
Step :  1
Implied Forward :  181.10401579
Implied Dividend Yield : 0.23878899%
Market American Call Price :  7.625
Binomial American Call Price :  7.62499999
Implied American Call Vol : 22.29131448%
Market American Put Price :  11.975
Binomial American Put Price :  11.97499988
Implied American Put Vol : 22.34350151%
Equivalent Binomial European Call Price :  7.62499999
Equivalent Binomial European Put Price :  11.46891026
-------------------------------------------------------
De-Americanization Results :
Step :  2
Implied Forward :  181.08206588
Implied Dividend Yield: 0.27362433%
Market American Call Price :  7.625
Binomial American Call Price :  7.62500002
Implied American Call Vol : 22.31540073%
Market American Put Price :  11.975
Binomial American Put Price :  11.97499991
Implied American Put Vol : 22.32333667%
Equivalent Binomial European Call Price :  7.62500002
Equivalent Binomial Europ

## Dividend Seasonality : Interpolation & Extrapolation

US companies typically pay cash dividends on a quarterly, semi-annual, or annual basis, with a noticeable seasonality in their dividend patterns. As a result, practitioners often forecast future dividend payment dates by assuming the same payment days and months as in previous years. A new "seasoned" dividend curve is then bootstrapped as follows:

Under the Non-Arbitrage assumption : $F_{T^{Exp}_n} = S_t × e^{r (T^{Exp}_n-t)} - ∑_{k ∈ [1,n]}d_k × e^{r(T^{Exp}_n-T_k)}$ 

Therefore :  $d_n = e^{-r\left(T^{Exp}_n-T_n\right)} × \left( S_t × e^{r \left(T^{Exp}_n-t\right)} - ∑_{k ∈ [1,n-1]}d_k × e^{r\left(T^{Exp}_n-T_k\right)} - F_{T^{Exp}_n} \right)$ 

And finally : $d_n = S_t × e^{r \left(T_n-t\right)} - ∑_{k ∈ [1,n-1]}d_k × e^{r\left(T_n-T_k\right)} - F_T × e^{-r(T-T_n)}$ 


Where  :

* $S_t$ the spot price at $t$
* $r$ the zero-coupon rate
* $\left(F_{T^{Exp}_n}\right)_n$ are the implied forwards at the market option expiries $\left(T^{Exp}_n\right)_n$, obtained by the de-Americanization method.
* $(d_k)_k$ are the expected cash dividends paid at the projected Ex-dividend dates $(T_k)_k$

On the other hand, practitioners typically assume continuous dividend yield curves for the indices. The most straightforward method to construct these curves is by assuming a piecewise constant curve between the market option expiries:

Under the Non-Arbitrage assumption : $F_{T^{Exp}_n} = S_t × e^{r \left(T^{Exp}_n-t\right) - ∑_{k ∈ [1,n]}q_k × \left(T^{Exp}_{k} - T^{Exp}_{k-1} \right)}$ 

Therefore : $q_n = ln\left( S_t × e^{r \left(T^{Exp}_n-t\right) - ∑_{k ∈ [1,n-1]}q_k × \left(T^{Exp}_{k} - T^{Exp}_{k-1} \right)} / F_{T^{Exp}_n}\right) / \left( T^{Exp}_n - T^{Exp}_{n-1}\right)$

Where  : $(q_k)_k$ are the piecewise constant instantaneous forward dividend yields between $\left( T^{Exp}_k - T^{Exp}_{k-1}\right)$

Cash and yield dividends can then be extrapolated with a flat yield after the last market option expiry.

## Implied European Volatilities

Once the forward and dividend curves are calibrated, the next step is to compute the implied European volatilities to price European payoffs on a U.S. underlying stock. To achieve this, we assume that the equivalent European Binomial prices are the market prices and use these to derive the implied European volatilities under the Black-Scholes standard market model (For each listed expiry):

Init :
- $F = F(d)$, $d$ and $F$ are the calibrated dividend yield and forward price by the above de-Americanization algorithm
- $Strikes$, the quoted strikes of OTM American call and put options

For $K$ in $Strikes$ :
- Compute the implied American Binomial volatility : $σ_{Amr}(K, F(d))$
- Compute the equivalent European Binomial price : $Binomial_{EUR}(K, σ_{Amr}(K, F(d)))$
- Calibrate the Implied European Volatilities $\sigma_{Imp}(K,T)$ such that :

$$
BS(F(d), K, \sigma_{Imp}(K,T), T) - Binomial_{EUR}(K, σ_{Amer}(K, F(d))) = 0  $$

The $\sigma_{Imp}(K,T)$ calibration is done via the Newton-Raphson method.



In [6]:
from scipy.stats import norm
from scipy import optimize

class bs_model:
    """
    The Black-Scholes model class.
    """
    def bs_price(self, S0, T, r, q, strike, vol, isCall) :
        """
        The European vanilla option Black price.
        """
        forward = S0 * np.exp((r - q) * T)
        v2T = vol**2 * T
        d1 = (np.log(forward / strike) + v2T / 2) / v2T**0.5
        d2 = d1 - v2T**0.5
        phi = 1 if isCall else  -1
        return np.exp(-r * T) * phi * (forward * norm.cdf(phi * d1) - strike * norm.cdf(phi * d2))
 
    def bs_vega(self, S0, T, r, q, strike, vol) :
        """
        The Black vega greek. 
        The derivative of the option value with respect to the volatility of the underlying asset.
        """
        forward = S0 * np.exp((r - q) * T)
        v2T = vol**2 * T
        d2 = (np.log(forward / strike) - v2T / 2) / v2T**0.5
        return np.exp(-r * T) * strike * norm.pdf(d2) * T**0.5
    
    def eur_implied_vol(self, S0, T, r, q, strike, isCall, target_price, maxiter=750, tol=1e-08) :
        """
        The Newton-Raphson algorithm : Implied European volatilities from market prices.
        """
        f = lambda vol : self.bs_price(S0, T, r, q, strike, vol, isCall) - target_price
        f_deriv = lambda vol : self.bs_vega(S0, T, r, q, strike, vol)
        return optimize.newton(f, 0.2, f_deriv, maxiter=maxiter, tol=tol)

bs_model = bs_model()

Let us get back our example of AAPL US stock options and calibrate its equivalent implied European volatilities :

In [7]:
amr_vols, eur_vols = [], []
for i in range(len(strikes)) :
    amr_vol = binomial_model.implied_vol(S0, T, r, div_yield, strikes[i], isCall[i], amer_prices[i])
    amr_vols.append(amr_vol)
    target_price = binomial_model.binomial_price(S0, T, r, div_yield, strikes[i], amr_vol, isCall[i], False)
    eur_vol = bs_model.eur_implied_vol(S0, T, r, div_yield, strikes[i], isCall[i], target_price)
    eur_vols.append(eur_vol)
    print("American implied vol at " + str(strikes[i]) + " : " + str(round(100 * amr_vol, 4)) + "%")
    print("European implied vol at " + str(strikes[i]) + " : " + str(round(100 * eur_vol, 4)) + "%")
    print("Difference : " + str(round(100 * abs(amr_vol - eur_vol), 4)) + "%")
    print("----------------------------------------")

American implied vol at 165 : 25.945%
European implied vol at 165 : 25.9505%
Difference : 0.0055%
----------------------------------------
American implied vol at 170 : 24.98%
European implied vol at 170 : 24.9814%
Difference : 0.0014%
----------------------------------------
American implied vol at 175 : 24.0761%
European implied vol at 175 : 24.0839%
Difference : 0.0078%
----------------------------------------
American implied vol at 180 : 23.0147%
European implied vol at 180 : 23.0174%
Difference : 0.0028%
----------------------------------------
American implied vol at 185 : 22.3197%
European implied vol at 185 : 22.3176%
Difference : 0.0022%
----------------------------------------
American implied vol at 190 : 21.6157%
European implied vol at 190 : 21.6129%
Difference : 0.0028%
----------------------------------------
